## Continuous Time RL
Markov decision processes and, thus, RL can be generalized to continuous-time settings.
The environment state and agent's action/control signal are continuous functions, $s(t)$ and  $a(t) \sim \pi(s(t))$, the discrete rewards are replaced by a reward rate function, $r(t) = r(s(t), a(t))$, and the state evolves according to a (potentially stochastic) state dynamics function, $\frac{ds(t)}{dt} = f(s(t),a(t))$.
Rather than maximizing the expected discounted sum of future rewards, the goal in continuous-time RL is maximizing the expected discounted \emph{integral} over the reward rate function,
\begin{align}
    \max_{\pi} \, \mathbb{E}_{\pi}\left [ \int_{t=0}^{\infty} \gamma^t r(t) dt \right ],
\end{align}
where $0 < \gamma <1$ is the discount per time unit. The definition of the value function in continuous-time \gls{rl} likewise replaces sums with integrals:
\begin{align}
     V(s) &=\mathbb{E}_{\pi}\left [ \int_{k=0}^{\infty} \gamma^{k} r(t+k) dk  | s(t) = s  \right ] 
\end{align}
Note that the optimal value function satisfies
\begin{align}
     V(s(t)) &= \max_{a} \mathbb{E}\left [ \int_{k=t}^{\infty} \gamma^{k-t} r(k) dk  \right ],\\
     &= \max_{a} \mathbb{E}\left [ \int_{k=t}^{t+\Delta t} \gamma^{k-t} r(k) dk + \gamma^{\Delta t}V(s(t+\Delta t))  \right ].
\end{align}
Taking the Taylor expansion of the term $V(s(t+\Delta t))$ and the limit as $\Delta t \rightarrow 0$, we arrive at the discounted version of the Hamilton–Jacobi–Bellman equation,
\begin{align}
    -\log(\gamma) V(s(t)) = \max_{a} \mathbb{E}\left [ r(s(t),a(t)) + \frac{\partial V(s(t))}{\partial s} \frac{ds(t)}{dt} \right ] .
\end{align}
% \gamma V(s(t)) = \max_{a} \expect*{ r(s(t),a(t)) + \frac{\partial V(s(t))}{\partial s} f(s(t),a(t))   }.
Thus, the following TD learning rule can be derived for this integral formulation of the value function:
\begin{align}
V(s(t)) \leftarrow V(s(t)) + \kappa \underbrace{\left [ r(t) + \log(\gamma) V(s(t)) + \frac{dV(s(t))}{dt} \right ]}_{=\delta^{(0)}(t)}.
\end{align}
We will refer to this as the continuous-time version of the TD(0) rule (Doya, 2000). The derivative of the value function can be approximated by a difference quotient. Counterparts to other TD rules can be derived as well. Using these, continuous-time value functions can be learned either in conjunction with a policy otherwise the gradient of the value function can be used to determinate optimal control/action signals.


While TD updates can drive value function learning in continuous-time, they cannot be applied to learn a useful Q-function. In continuous-time, the Q-function collapses to the value function, and, thus, is problematic as a tool for decision making (Tallec, 2019). One can define a Q-function with fine time discretization, but the result is highly sensitive to the size of the time-step (Tallec, 2019). Alternatively, actions can be restricted to a Lipschitz continuous function (i.e., $|\frac{da(t)}{dt}| \leq L$), in which case the Q-function is well-defined (Kim, 2021).

## Memory in RL

Often in RL, a replay memory is used to store \emph{experiences} -- samples of past states, actions, and rewards, $(s_t,s_{t+1},r_t,a_t)$ at every time-step. This memory is repeatedly sampled from to compute TD errors (e.g., TD(0), TD(n) or offline TD($\lambda$) / $\lambda$-return) and train the model. The number of experiences stored in the replay memory may become quite large. This approach is not suitable for biologically plausible RL models. Neural circuits do not have perfect, long lists of memories. Additionally, it is not suitable when dealing with RL in continuous time as there are no discrete time-steps to differentiate experiences and ieven if time is finely discretized this would result in very large memory requirements. 

One way to deal with this is to use eligibility traces, a method for online TD learning. Instead of updating value estimates based on future information, current information is used to update past value estimates. This is done by maintaining a vector, the eligibility trace, that holds the decaying values of $V(s)$. This restricts the type of TD update performed though and in practice experience reply techniques can result in better performance than eligibility traces in case of continuous state spaces.

Here we will develop new methods for online, on-policy TD learning in spiking neural networks that operate in continuous-time and use biologically plausible memory mechanisms bu using LMUs. The LMU allow us to encode windowed histories of a continuous signals. Given an LMU memory state, $\mathbf{m}(t)$, computed from input signal, $u(t)$, we can linearly decode not only delays, $u(t - k)\approx \mathbf{P}\left (\frac{k}{\theta} \right )\mathbf{m}(t)$, but any linear function of the signal history -- including continuous-time TD errors. Additionally, we can obtain approximations of past neural activity to update past estimates using current memory vectors (somewhat like standard eligibility traces).

<!-- There has been past work on implementing RL algorithms in continuous time with spiking neural networks. However, obtaining the past activity of spiking neurons for such updates is a challenge.
The Actor-Critic (AC) model in Fremaux (2013) does not actually compute TD signals with spiking neurons to avoid this.
The hierarchical reinforcement learning (HRL) model in Rasmussen (2017) addresses the challenge by using two identical neural populations to represent current and delayed Q functions, with mechanisms for copying learned weights from one population to the other.  This is similar to a TD(0) algorithm with a target network. The delay used is fixed in advance. Different tasks may require credit assignment over time windows of different lengths and, in many cases, better performance can be achieved by using reward information over many time steps for updates.
While in Zambrano (2015), a continuous-time version of on-policy SARSA was presented, which used a winner-take-all network for discrete action selection and a working memory that stores a decaying race of neural activities required for learning. -->


## Nengo
To move towards biologically constrained/plausible RL we shall build neural networks using the Nengo package. Nengo models can be simulated with a small timestep so we can model continuous time RL. Nengo networks consist of 
* Node: This stores/outputs a vector over the simulation run time. Often used to provide input to the ensembles
* Ensemble: A group of (spiking) neurons that collectively represent a vector (the latent representation)
* Connection(pre, post): Used to connect nodes, ensembles, and other network components. If the pre object in the connection is an ensemble Nengo will create a decoded connection. When the pre object is anything else, Nengo will create a direct connection. The post object is only used to determine which signal will receive the data produced by the connection.

Check out https://www.nengo.ai to learn more.

### Decoded connection
The important thing about decoded connections is that they do not directly compute the function defined for that connection (keeping in mind that passing in no function is equivalent to passing in the identity function). Instead, the function is approximated by solving for a set of decoding weights. The output of a decoded connection is the sum of the pre ensemble’s neural activity weighted by the decoding weights solved for in the build process.

Mathematically, you can think of a decoded connection as implementing the following equation:
$$ y(t) = \sum_{i=0}^{N} \mathbf{d}^f_i a_i(x(t))$$
where
* $y(t)$ is the output of the connection at time $t$
* $N$ is the number of neurons in the pre ensemble
* $\mathbf{d}_i$ is the decoding weight associated with neuron $i$. If you set the connection  $f$
* $a_i(x(t))$ is the activity of neuron $i$ given $x(t)$, the input at time $t$.

Decoders will be automatically solved for in the Nengo build process. They are optimized to approximate the function or transformation or data points supplied when defining the Connection object. But decoders can also be learned online...

## PES learning rule
The Prescribed Error Sensitivity  is a biologically plausible supervised learning rule.
To learn a connection between a pre-population of $N$ neurons and a post-population of neurons encoding a $n-$dim vector, this rule modifies the pre-population's decoders in response to an error signal:
\begin{align}
    \Delta D = \kappa \mathbf{\delta}(t) \mathbf{a}(t)^T
\end{align}
where $\kappa$ is a learning rate, $D \in \mathbb{R}^{n \times N}$ is the matrix of decoders, $\mathbf{a}(t) \in \mathbb{R}^{N \times 1}$ is the vector of (filtered) neural activites, $\mathbf{\delta}(t) \in \mathbb{R}^{n \times 1}$ is the error. (for value learning we'll have $n=1$)


The error signal may be computed by other neural populations in a model. Biologically, we can think of those populations as dopaminergic neurons that can modify weights in this way via dopamine levels.

See https://www.nengo.ai/nengo/examples/learning/learn-communication-channel.html for an example using PES

## LMUs
An LMU memory state, $\mathbf{m} \in \mathbb{R}^{q}$, for an order $q$ Legendre representation, is updated according to 
\begin{equation}
    \dot{\mathbf{m}}(t) =  A\mathbf{m}(t) +  Bu(t),
\end{equation}

where $u(t)$ is the input signal.  Where $A$ and $B$ are given by 
\begin{equation}
    A_{ij} =\frac{(2i + 1)}{\theta} \begin{cases}
        -1 & i < j \\
        (-1)^{i-j+1} & i \geq j
    \end{cases}, \quad B_{i} = \frac{(2i + 1)(-1)^{i}}{\theta}.
    \end{equation}
    
   where $\theta$ is the time window of the memory.
    
   An approximation of the input in the past, at, say $\tau\theta$ seconds in the past ($0\leq \tau \leq 1$) is given by 
   
   $$u(t-\tau\theta) \approx \mathbf{P}^q(\tau)\mathbf{m}(t)$$
   
   where $\mathbf{P}^{q_a}(\tau) \in \mathbb{R}^{1 \times q}$ is the vector of the shifted Legendre polynomials (of degree one to $q$), evaluated at $\tau$.
   
   
   If we want to remember a vector signal $\mathbf{u}(t) \in \mathbb{R}^n$ instead of a scalar signal, we can use multiple LMUs, all stacked together in a matrix, $\mathbf{m} \in \mathbb{R}^{q \times n}$:
  \begin{equation}
    \dot{\mathbf{m}}(t) =  A\mathbf{m}(t) +  \begin{bmatrix} B , \dots , B \end{bmatrix} \mathbf{u}(t),
\end{equation} 

See the notebook "LMU with changing theta" for LMU examples

# RL with LMUs
For now, let's just consider the problem of training a critic network. Actions will come from a fixed pre-defined schedule. We have a neural population representing the state $s(t) \in \mathbb{R}^d$. A value neural population recieves the state as input and (after training) encodes the value function, $V(s(t))\in \mathbb{R}$. 

We will use the PES learning rule with the error signal being a TD error to train the decoders on the connection between the state population and value population. This will require memory --not just to compute the TD error, but also to recall state activities for the $\mathbf{a}(t)$ term in the PES rule.

Let $\mathbf{m}_V(t) \in \mathbb{R}^{q_v }$ be a LMU memory of the value output, and $\mathbf{m}_R(t) \in \mathbb{R}^{q_r }$ be a LMU memory of the reward signal.
Let $\mathbf{m}_a(t) \in \mathbb{R}^{q_a \times N}$ be a LMU stack remembering the state population's activities. Note that unlike the last two, this is a memory of $N$ signals, so $\mathbf{m}_a(t)$ is a matrix. Assume all LMUs have a time window of $\theta$.


The PES rule to update a past value estimate (the one $\theta$ seconds ago) given TD error $\delta(t)$, is 
\begin{align} 
    \Delta \mathbf{d} = \kappa \delta(t)\mathbf{P}^{q_a}(1) \mathbf{m}_{a_j}(t),
\end{align}


#### TD(0)

Recall that the continuous-time version of the TD(0) error is given by $\delta^{(0)}(t) = r(t) + \log(\gamma) V(s(t)) + \frac{dV(s(t))}{dt}$. Let $\mathbf{m}_r(t) \in \mathbb{R}^{q_r}$ be an LMU memory of the reward signal and $\mathbf{m}_v \in \mathbb{R}^{q_v }$ be an LMU memory of the output of the value population. Assume all LMUs have a time window of $\theta$. We can decode the TD(0) error for our estimate $t-k$ seconds in the past from these memory traces:
\begin{align}
\delta^{(0)}(t-k) &= r(t-k) + \log(\gamma) V(s(t-k)) - \frac{dV(s(t-k))}{dk} \\
&= \mathbf{P}^{q_r}\left (\frac{k}{\theta} \right  )\mathbf{m}_r(t) + \left [\log(\gamma) \mathbf{P}^{q_v}\left (\frac{k}{\theta} \right  ) - \frac{d\mathbf{P}(k/\theta)}{dk} \right ] \mathbf{m}_v(t)
\end{align}
To train a critic network with this error, we can replay through a $\theta$ second long history of the state via an \gls{lmu}, $\mathbf{m}_s(t)\in \mathbb{R}^{q_s \times d}$. This would require interweaving periods of experience gathering (during which the LMU memory traces are built through recurrent dynamics) with replay and learning (during which the LMU states are frozen and sampled). Replay through state memories can be done with forward replay, backward replay, or via random sampling. 

This TD(0) error could be used to drive any error-based learning algorithm. We will use it to train a spiking neural network with the biologically plausible PES learning rule. Let $T$ be end time of the last experience-gathering phase. The decoder on the $i^{\text{th}}$ neuron of the state population will be updated via
\begin{align}
    \Delta \mathbf{d}_i(t) &= \kappa \delta^{(0)}(T- k(t)\theta) a_i(t) \\
    &= \kappa \left [ \mathbf{P}^{q_r}\left (k(t) \right  )\mathbf{m}_r(t) + \left (\log(\gamma) \mathbf{P}^{q_v}\left (\tau(t) \right  ) - \frac{1}{\theta}\frac{d\mathbf{P}(k(t))}{dk} \right ) \mathbf{m}_v(t) \right ] a_i(t),\\ 
    & \quad\quad\quad \text{while } s(t) =  \mathbf{P}^{q_s}(k(t))\mathbf{m}_s(t), 
\end{align}
where $k(t) \in [0,1]$ determines the replay mechanism: $k(t)=0$ corresponds to sampling the most recent memory from the LMU (the state, reward, and value estimate at time $T$) while $k(t)=1$ corresponds to sampling the most distant memory from the LMU (the state, reward, and value estimate at time $T-\theta$). Forward replay over a learning time window of length $w$, would obtained by $k(t) = \frac{w-t}{w-T}$, while backward replay occurs when  $k(t) = \frac{t-T}{w-T}$. 

See the notebook 'TD(theta) - learning with replay' for an example of this.

By separating experience and learning phases we are learning offline. To avoid the need for replay and to have fully online learning, we require an additional memory unit. Assume we have a population of $N$ neurons representing the state.  We need an LMU, $\mathbf{m}_n(t)\in \mathbb{R}^{q_n \times N }$, that encodes the history of the state population's neural activities/ spike trains. With this we can compute the updates to the decoders without explicit stepping through the memory of the state signal,
\begin{align}
    \Delta \mathbf{d}_i(t) &= \kappa \delta^{(0)}(t - \theta)  \mathbf{P}^{q_n}(1)\mathbf{m}_n(t) \\
    &= \kappa \left [ \mathbf{P}^{q_r}(1)\mathbf{m}_r(t) + \left (\log(\gamma) \mathbf{P}^{q_v}(1) - \frac{1}{\theta} \left. \frac{d\mathbf{P}(k)}{dk} \right |_{k=1} \right ) \mathbf{m}_v(t) \right ]\mathbf{P}^{q_n}(1)\mathbf{m}_n(t).
\end{align}
This is slightly different from the standard PES learning rule -- rather than update weights using an error signal and the current local activity, we update using an error signal and an external activity memory signal.
In this version of TD(0), value estimates from $\theta$ seconds in the past are continuously updated, while the agent concurrently interacts with the environment and obtains new experiences. In this formulation, the LMUs are never frozen and the matrices applied to the LMU are fixed and so can be pre-computed and used to set weights between the recurrent LMU populations and the population encoding the TD error signal. In practice, a mix of both styles of learning (continuously online \& with replay) could be used.



### Reward normalization
A possible issue with this is the magnitude of the value function. A neural population represents the value function. It can only represent variables between -1 and 1 so if the value function exceeds that, the population will 'saturate'. To avoid this, assume that the reward is bounded, $|R(s)| \leq r_{max}$ for all states $s$. Then since $V(s) = \mathbb{E} [ \int_{0}^{T} \gamma^t r(s(t)) dt | s(0) = s]$,
$$\rightarrow |V| \leq r_{max} \int_{0}^{T} \gamma^t dt = r_{max} \frac{\gamma^T - 1}{\log(\gamma)}$$
If we want $ |V| \leq 1 $ then normalize rewards so that $r_{max} \leq \frac{\log(\gamma)}{\gamma^T - 1}$. Note that there may be better ways to normalize/modify either rewards, discount rates, or error signals to prevent saturation. In fact, if the reward is often less than $r_{max}$ (e.g., a sparse reward problem) then this normalization may result in a value function with very low magnitude. This may cause problems when using spiking neural networks to represent and compare values